In [1]:
from datetime import datetime
import re
import json

import numpy as np
from pandas import DataFrame, read_csv, unique

In [2]:
dateparse = lambda d: datetime.strptime(d, '%Y, %m, %d')
pw_df = read_csv(
    "../fast_place_weather.csv", 
    encoding="cp932",  
    header=[0,1,2,3], 
    index_col=[0,1], 
    parse_dates=[0], 
    date_parser=dateparse)

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (6,7,13,15,34,35,42,43,48,49,56,57,61,62,63,65,67,69,70,71,76,77,85,89,90,91,92,93,95,98,99,126,145,146,147,149,151,154,155,160,161,162,168,169,188,189,190,191,193,196,197,202,203,207,211,218,230,231,238,239,257,258,259,260,261,263,265,266,267,271,272,273,275,277,279,281,286,287,295,308,314,315,323,336,342,343,344,350,351,364,392,406,425,426,427,431,434,435,454,455,463,476,481,482,483,484,487,489,490,491,496,497,505,509,510,511,512,515,517,518,519,532,538,539,540,546,547,552,553,561,574,580,581,582,588,589,594,595,602,603,621,622,623,624,625,627,631,663,664,665,667,669,672,673,686,705,706,707,708,709,711,713,714,715,728,762,763,769,770,771,783,803,804,805,811,812,813,818,819,827,831,832,833,837,840,841,845,846,847,849,851,853,854,855,859,860,861,862,863,865,868,869,873,874,875,876,877,881,882,883,888,889,897,902,903,909,911,918,924,930,931,939,943,952,974,986,987,995,999,1000,1001,1005,1008,1009

In [6]:
ENTRY_DICT = {
    "weather_entry": "天気",
    "total_cloud_cover_entry": "雲量",
    "ceiling_entry": '雲底高度',
    "temp_entry": '気温(℃)',
    "humidity_entry": '湿度(%)',
    "dp_temp_entry": '露点温度(℃)',
    "precip_entry": '3時間降水量',
    "pressure_change_entry": '現地気圧(hPa)',
    "pressure_entry": '現地気圧(hPa)(変化量)',
    "sea_level_pressure_entry": '海面気圧(hPa)',
    "wind_direction_entry": '風向(16方位)',
    "wind_speed_entry": '風速(m/s)',
    "visibility_entry": '視程(km)',
    "discomfort_index_entry": '不快指数',
}
list(ENTRY_DICT.values())

['天気',
 '雲量',
 '雲底高度',
 '気温(℃)',
 '湿度(%)',
 '露点温度(℃)',
 '3時間降水量',
 '現地気圧(hPa)',
 '現地気圧(hPa)(変化量)',
 '海面気圧(hPa)',
 '風向(16方位)',
 '風速(m/s)',
 '視程(km)',
 '不快指数']

In [7]:
["sdfdf"]*5

['sdfdf', 'sdfdf', 'sdfdf', 'sdfdf', 'sdfdf']

In [3]:
pw_df.head()

北海道地方                                                  \
                   道北                                                   
                   旭川                                                   
                   天気    雲量 雲底高度 気温(℃) 湿度(%) 露点温度(℃) 3時間降水量 現地気圧(hPa)   
date       hour                                                         
2016-01-01 3     にわか雪  9割以上  ---  -4.4    89    -5.9    0.5    1001.1   
           6     にわか雪  9割以上  ---  -4.5    94    -5.3    0.0    1001.7   
           9     にわか雪  9割以上  ---  -4.1    87    -5.9    0.5    1003.1   
           12    にわか雪  9割以上  ---  -3.0    78    -6.3    0.0    1003.3   
           15      晴れ    6割  ---  -3.0    71    -7.5    0.0    1004.0   

                                         ...   沖縄地方                           \
                                         ...    沖縄県                            
                                         ...   南大東島                            
                現地気圧(hPa)(変化量) 海面気圧(hPa) ...  湿度(%) 露点温度(℃) 3時間降水量 現地気圧(hPa)   
date       hour                          ...                                   
2016-01-01 3             (---)    1019.1 ...     62    11.5    0.0    1023.8   
           6            (+0.6)    1019.7 ...     54     8.6    0.0    1024.7   
           9            (+1.4)    1021.1 ...     63    12.3    0.0    1026.0   
           12           (+0.2)    1021.2 ...     52    11.0    0.0    1025.0   
           15           (+0.7)    1021.9 ...     55    11.2    0.0    1023.6   

                                                                       
                                                                       
                                                                       
                現地気圧(hPa)(変化量) 海面気圧(hPa) 風向(16方位) 風速(m/s) 視程(km) 不快指数  
date       hour                                                        
2016-01-01 3            (-0.2)    1026.3        北       5     15   63  
           6            (+0.9)    1027.2      北北東       5      7   62  
           9            (+1.3)    1028.5       北東       6      9   65  
           12           (-1.0)    1027.5      北北東       6     15   66  
           15           (-1.4)    1026.1       北東       6     15   65  

[5 rows x 2156 columns]

In [60]:
pw_cols_lvls = pw_df.columns.levels
# pw_cols_lvls[1].tolist()

pw_cols_lvls

FrozenList([['中国地方', '九州地方', '北海道地方', '北陸地方', '四国地方', '東北地方', '東海地方', '沖縄地方', '近畿地方', '関東・甲信地方'], ['三重県', '京都府', '佐賀県', '兵庫県', '千葉県', '和歌山県', '埼玉県', '大分県', '大阪府', '奈良県', '宮城県', '宮崎県', '富山県', '山口県', '山形県', '山梨県', '岐阜県', '岡山県', '岩手県', '島根県', '広島県', '徳島県', '愛媛県', '愛知県', '新潟県', '東京都', '栃木県', '沖縄県', '滋賀県', '熊本県', '石川県', '神奈川県', '福井県', '福岡県', '福島県', '秋田県', '群馬県', '茨城県', '道北', '道南', '道央', '道東', '長崎県', '長野県', '青森県', '静岡県', '香川県', '高知県', '鳥取県', '鹿児島県'], ['むつ', '三宅島', '三島', '上野', '下関', '与那国島', '久米島', '京都', '人吉', '仙台', '伊良湖', '伏木', '佐世保', '佐賀', '倶知安', '八丈島', '八戸', '函館', '前橋', '勝浦', '北見枝幸', '千葉', '南大東島', '南鳥島', '厳原', '名古屋', '名瀬', '名護', '呉', '和歌山', '四日市', '境', '多度津', '大分', '大島', '大船渡', '大阪', '奈良', '姫路', '宇和島', '宇都宮', '室戸岬', '室蘭', '宮古', '宮古島', '宮崎', '宿毛', '富山', '寿都', '小名浜', '小樽', '尾鷲', '屋久島', '山口', '山形', '岐阜', '岡山', '岩見沢', '帯広', '平戸', '広尾', '広島', '延岡', '彦根', '御前崎', '徳島', '敦賀', '新庄', '新潟', '日光', '日田', '旭川', '札幌', '東京都心', '松山', '松本', '松江', '枕崎', '根室', '横浜', '水戸', '江差', '沖永良部', '河口湖', '油津', '津', '津山', 

In [61]:
# pw_df['北海道地方', '道北', '旭川', '天気']['2016-01-01':'2016-01-05']

In [62]:
si_df = read_csv("store_info.csv")

In [191]:
# si_df["area_name"]

train_area_df = DataFrame(
    si_df['area_name'].str.split(' ', 2).tolist(), 
    columns=['big_area', 'mid_area', 'sml_area']
)
train_area_df = train_area_df.drop("sml_area", axis=1)
train_area_df = train_area_df.applymap(lambda area: area.split('-', 1)[0])
train_area_df['big_area'].unique()

array(['Tokyo', 'Hyogo', 'Fukuoka', 'Hiroshima', 'Osaka', 'Shizuoka',
       'None', 'Kanagawa', 'Hokkaido', 'Miyagi', 'Niigata', 'Saitama'], dtype=object)

In [64]:
arr = train_area_df['mid_area'].unique()

In [188]:
ac_df = read_csv("../area_code.csv", encoding="cp932")
search_col = ac_df['big_area']
ac_df[search_col == '北海道地方']
# ac_df['sml_area'].unique()

,big_area,mid_area,sml_area,place_code
0,北海道地方,道北,旭川,1/1/47407
1,北海道地方,道北,稚内,1/1/47401
2,北海道地方,道北,北見枝幸,1/1/47402
3,北海道地方,道北,羽幌,1/1/47404
4,北海道地方,道北,留萌,1/1/47406
5,北海道地方,道東,釧路,1/3/47418
6,北海道地方,道東,根室,1/3/47420
7,北海道地方,道東,網走,1/3/47409
8,北海道地方,道東,雄武,1/3/47405
9,北海道地方,道東,帯広,1/3/47417


In [190]:
# Open si2pw_area_translate.json
with open("si2pw_area_translate.json", "rb") as f:
    content = f.read().decode("utf-8")
    s2p_trans = json.loads(content)
    print(s2p_trans)

{'Numazu': ['東海地方', '静岡県', '三島'], 'Shimada': ['東海地方', '静岡県', '御前崎'], 'Ito': ['東海地方', '静岡県', '網代'], 'Fujinomiya': ['東海地方', '静岡県', '静岡'], 'Nishinomiya': ['近畿地方', '兵庫県', '神戸'], 'Amagasaki': ['近畿地方', '大阪府', '大阪'], 'Akashi': ['近畿地方', '兵庫県', '神戸'], 'Kakogawa': ['近畿地方', '兵庫県', '姫路'], 'Takarazuka': ['近畿地方', '大阪府', '大阪'], 'Tokachi': ['北海道地方', '道東', '釧路'], 'Ashibetsu': ['北海道地方', '道北', '旭川'], 'Kato': ['北海道地方', '道東', '帯広'], 'Kurume': ['九州地方', '佐賀県', '佐賀'], 'Chikushino': ['九州地方', '福岡県', '福岡'], 'Kitakyushu': ['中国地方', '山口県', '下関'], 'Itoshima': ['九州地方', '福岡県', '福岡'], 'Yame': ['九州地方', '佐賀県', '佐賀'], 'Nagaoka': ['北陸地方', '新潟県', '新潟'], 'Joetsu': ['北陸地方', '新潟県', '高田'], 'Shibata': ['北陸地方', '新潟県', '新潟'], 'Kashiwazaki': ['北陸地方', '新潟県', '高田'], 'osaki': ['東北地方', '宮城県', '石巻'], 'Shiogama': ['東北地方', '宮城県', '仙台'], 'Niiza': ['関東・甲信地方', '東京都', '東京都心'], 'Higashihiroshima': ['中国地方', '広島県', '呉'], 'Hiroshima': ['中国地方', '広島県', '広島'], 'Machida': ['関東・甲信地方', '神奈川県', '横浜'], 'Setagaya': ['関東・甲信地方', '東京都', '東京都心'], 'Chuo': ['関東

In [189]:
# Oepn si_big_area_translate.json
with open("si_big_area_translate.json", "rb") as f:
    content = f.read().decode("utf-8")
    sba_trans = json.loads(content)
    print(sba_trans)

{'Tokyo': ['関東・甲信地方', '東京都'], 'Hyogo': ['近畿地方', '兵庫県'], 'Fukuoka': ['九州地方', '福岡県'], 'Hiroshima': ['中国地方', '広島県'], 'Osaka': ['近畿地方', '大阪府'], 'Shizuoka': ['東海地方', '静岡県'], 'Kanagawa': ['関東・甲信地方', '神奈川県'], 'Hokkaido': ['北海道地方', 'None'], 'Niigata': ['北陸地方', '新潟県'], 'Saitama': ['関東・甲信地方', '埼玉県'], 'None': ['None', 'None']}
